In [38]:
import pandas as pd
import ftfy
import streamlit as st
from pymongo import MongoClient
import time
from datetime import datetime
import pandas as pd
from openai import OpenAI
import streamlit.components.v1 as components
import sys
import json
from navigation import render_nav

In [39]:
# df = pd.read_csv("final_user_batches.csv", encoding="utf-8")
# df["abstract_fixed"] = df["abstract"].apply(fix_text)

In [40]:
# bad = df.loc[df["abstract_id"] == 1045, "abstract"].iloc[0]
# print(bad)
# print(bad.encode("utf-8", errors="backslashreplace"))
# print(bad.encode("latin1", errors="backslashreplace"))

In [41]:
# df = pd.read_csv("final_user_batches.csv")

In [42]:
# print(df.isna().sum())

In [43]:
# print(df.columns.tolist())

In [44]:
# df.loc[df["user_id"] == "jchan51@illinois.edu"]

In [ ]:
def get_openai_client():
    return OpenAI()

client_openai = get_openai_client()

In [46]:
def format_sata(sata_list):
    out = []
    for i, q in enumerate(sata_list, 1):
        out.append(f"SATA {i}: {q['question']}")
        for j, opt in enumerate(q["options"], 1):
            out.append(f"  {j}. {opt}")
        out.append(f"Correct answers: {', '.join(q['correct_answers'])}")
        out.append("")
    return "\n".join(out)

In [47]:
# # conversation_text = (
# #     "User: What Gamma aminobutyric acid (GABA) receptor agonists? "
# #     "Assistant: Gamma aminobutyric acid (GABA) receptor agonists are substances that bind to and activate "
# #     "GABA receptors in the brain. GABA is the primary inhibitory neurotransmitter in the central nervous system, "
# #     "meaning it tends to reduce neuronal excitability. By activating GABA receptors, these agonists enhance the "
# #     "inhibitory effect of GABA, generally leading to a calming effect on the nervous system. This action can make "
# #    ? receptor agonists being studied for their potential neuroprotective effects and role in "
# #     "treating acute stroke. However, the studies highlighted seem to indicate that while there might be a "
# #     "theoretical basis for their use in stroke treatment, practical application is limited by side effects such as "
# #     "somnolence (drowsiness) and rhinitis (inflammation of the nasal mucous membrane). "
    
# #     "User: What is total anterior circulation syndrome? "
# #     "Assistant: Total Anterior Circulation Syndrome (TACS) is one of the classifications used to describe a "
# #     "clinical pattern of stroke, specifically affecting the anterior part of the brain's blood supply. It is part "
# #     "of the Oxford Community Stroke Project (OCSP) classification, which categorizes strokes based on the symptoms "
# #     "they produce and the areas of the brain affected. TACS is associated with a blockage or reduced blood flow in "
# #     "the main arteries supplying the front part of the brain, particularly the middle cerebral artery and sometimes "
# #     "parts of the anterior cerebral artery. This type of stroke typically results in a more extensive area of brain "
# #     "infarction. The classic symptoms of TACS include weakness or sensory loss on one side of the body, higher "
# #     "cortical dysfunction such as speech problems or spatial neglect, and loss of half of the visual field in both "
# #     "eyes on the same side. A diagnosis of TACS generally implies a large stroke with significant disability and a "
# #     "poorer prognosis compared to other stroke subtypes. In research, understanding TACS is important because "
# #     "treatment response and outcomes can differ by stroke subtype. "
    
# #     "User: What is chlormethiazole? "
# #     "Assistant: Chlormethiazole is a sedative and hypnotic drug that acts primarily as a GABA receptor agonist. By "
# #     "activating GABA receptors, it enhances the brain’s main calming system. It has been used for sedation, "
# #     "including in insomnia and alcohol withdrawal. It has also been studied for possible neuroprotective effects "
# #     "in acute stroke. Its major side effects include somnolence (strong drowsiness) and rhinitis (nasal "
# #     "inflammation), which limit its use, especially in stroke where too much sedation can be harmful."

# #     "User: How is Medline involved in this study? "
# #     "Assistant:"
# #     "MEDLINE is involved in this study only as a literature source, not as part of the experiment itself."

# #     "The authors were conducting a systematic review, so they needed to find all existing studies about GABA receptor agonists for acute stroke. To do this, they searched several medical databases, including MEDLINE, Embase, CENTRAL, and Chinese databases."

# #     "MEDLINE’s role was to serve as a database of published medical research. The researchers used it to locate randomized controlled trials that studied GABA receptor agonists in patients with acute stroke and reported outcomes like death, disability, and side effects."

# #     "In summary, MEDLINE did not produce new data for this study. It was used to find previously published studies that were then analyzed in the Cochrane Review."

# #     "User: Explain Placebo?"
# #     "Assistant: A placebo is something that looks like a real medical treatment but has no active medical ingredient or therapeutic effect."
# #     "It’s often used in clinical trials to see whether a new drug or treatment actually works better than “doing nothing” or better than expectations alone."
# # )

# conversation_text = (
#     "User: What is EGFR mutation–positive (M+) non-small cell lung cancer, and how is it different from other types of lung cancer?\n"
#     "User: Why are EGFR M+ tumors more common in women and people who do not smoke?\n"
#     "User: What treatments were compared in this review for first-line therapy of EGFR M+ NSCLC?\n"
#     "User: What were the main outcomes the researchers were trying to measure?\n"
#     "User: How many clinical trials and patients were included in this analysis?\n"
#     "User: Which EGFR-targeted drugs were most commonly studied?\n"
#     "User: What did the results show about progression-free survival when using TKIs compared to chemotherapy?\n"
#     "User: Did these treatments improve overall survival, and why might the results be unclear?\n"
#     "User: What side effects were most common with EGFR-targeted therapies compared to chemotherapy?\n"
#     "User: How did these treatments affect quality of life?\n"
#     "User: Is combining a TKI with chemotherapy better than using a TKI alone?\n"
#     "User: What is the current standard of care for patients with EGFR M+ NSCLC?\n"
#     "User: Why is it hard to interpret overall survival results in these trials?\n"
#     "User: Are monoclonal antibody therapies useful for this type of lung cancer?\n"
#     "User: Why is icotinib not commonly used outside of China?\n"
# )

# sata_text = [
#     {
#         "question": "What was the primary focus of the study regarding EGFR M+ NSCLC treatments?",
#         "options": [
#             "To determine if EGFR-targeted drugs improve progression-free survival",
#             "To investigate the genetic causes of NSCLC",
#             "To compare different surgical options for EGFR-positive lung cancer"
#         ],
#         "correct_answers": [
#             "To determine if EGFR-targeted drugs improve progression-free survival"
#         ]
#     },
#     {
#         "question": "How might the small sample sizes of certain trials affect the validity of conclusions drawn regarding the efficacy of combining TKIs and chemotherapy?",
#         "options": [
#             "It strengthens the reliability of conclusions",
#             "It limits the generalizability of the findings",
#             "It ensures consistency in treatment outcomes"
#         ],
#         "correct_answers": [
#             "It limits the generalizability of the findings"
#         ]
#     },
#     {
#         "question": "Critically assess why the study concludes that TKIs improve health-related quality of life compared to chemotherapy.",
#         "options": [
#             "TKIs are shown to delay cancer progression and have fewer side effects",
#             "TKIs consistently improve overall survival in all trials",
#             "Monoclonal antibodies have proven superior to TKIs"
#         ],
#         "correct_answers": [
#             "TKIs are shown to delay cancer progression and have fewer side effects"
#         ]
#     },
#     {
#         "question": "Which EGFR-targeted drugs were found to delay cancer progression in the study?",
#         "options": [
#             "Erlotinib",
#             "Gefitinib",
#             "Cetuximab"
#         ],
#         "correct_answers": [
#             "Erlotinib",
#             "Gefitinib"
#         ]
#     }
# ]


# abstract = """Epidermal growth factor receptor (EGFR) mutation positive (M+) non-small cell lung cancer (NSCLC) is an important subtype of lung cancer comprising 10% to 15% of non-squamous tumours. This subtype is more common in women than men, is less associated with smoking, but occurs at a younger age than sporadic tumours.

# To assess the clinical effectiveness of single-agent or combination EGFR therapies used in the first-line treatment of people with locally advanced or metastatic EGFR M+ NSCLC compared with other cytotoxic chemotherapy (CTX) agents used alone or in combination, or best supportive care (BSC). The primary outcomes were overall survival and progression-free survival. Secondary outcomes included response rate, symptom palliation, toxicity, and health-related quality of life.

# We conducted electronic searches of the Cochrane Register of Controlled Trials (CENTRAL) (2020, Issue 7), MEDLINE (1946 to 27th July 2020), Embase (1980 to 27th July 2020), and ISI Web of Science (1899 to 27th July 2020). We also searched the conference abstracts of the American Society for Clinical Oncology and the European Society for Medical Oncology (July 2020); Evidence Review Group submissions to the National Institute for Health and Care Excellence; and the reference lists of retrieved articles.

# Parallel-group randomised controlled trials comparing EGFR-targeted agents (alone or in combination with cytotoxic agents or BSC) with cytotoxic chemotherapy (single or doublet) or BSC in chemotherapy-naive patients with locally advanced or metastatic (stage IIIB or IV) EGFR M+ NSCLC unsuitable for treatment with curative intent.

# Two review authors independently identified articles, extracted data, and carried out the 'Risk of bias' assessment. We conducted meta-analyses using a fixed-effect model unless there was substantial heterogeneity, in which case we also performed a random-effects analysis as a sensitivity analysis.

# Twenty-two trials met the inclusion criteria. Ten of these exclusively recruited people with EGFR M+ NSCLC; the remainder recruited a mixed population and reported results for people with EGFR M+ NSCLC as subgroup analyses. The number of participants with EGFR M+ tumours totalled 3023, of whom approximately 2563 were of Asian origin.

# Overall survival (OS) data showed inconsistent results between the included trials that compared EGFR-targeted treatments against cytotoxic chemotherapy or placebo. Erlotinib was used in eight trials, gefitinib in nine trials, afatinib in two trials, cetuximab in two trials, and icotinib in one trial. The findings of FASTACT 2 suggested a clinical benefit for OS for participants treated with erlotinib plus cytotoxic chemotherapy when compared to cytotoxic chemotherapy alone, as did the Han 2017 trial for gefitinib plus cytotoxic chemotherapy, but both results were based on a small number of participants (n = 97 and 122, respectively).

# For progression-free survival (PFS), a pooled analysis of four trials showed evidence of clinical benefit for erlotinib compared with cytotoxic chemotherapy (hazard ratio (HR) 0.31; 95% confidence interval (CI) 0.25 to 0.39; 583 participants; high-certainty evidence). A pooled analysis of two trials of gefitinib versus paclitaxel plus carboplatin showed evidence of clinical benefit for PFS for gefitinib (HR 0.39; 95% CI 0.32 to 0.48; 491 participants; high-certainty evidence), and a pooled analysis of two trials of gefitinib versus pemetrexed plus carboplatin with pemetrexed maintenance also showed evidence of clinical benefit for PFS for gefitinib (HR 0.59; 95% CI 0.46 to 0.74; 371 participants; moderate-certainty evidence). Afatinib showed evidence of clinical benefit for PFS when compared with chemotherapy in a pooled analysis of two trials (HR 0.42; 95% CI 0.34 to 0.53; 709 participants; high-certainty evidence).

# All but one small trial showed a corresponding improvement in response rate with tyrosine-kinase inhibitor (TKI) compared to chemotherapy. Commonly reported grade 3/4 adverse events associated with afatinib, erlotinib, gefitinib and icotinib monotherapy were rash and diarrhoea. Myelosuppression was consistently worse in the chemotherapy arms; fatigue and anorexia were also associated with some chemotherapies.

# Seven trials reported on health-related quality of life and symptom improvement using different methodologies. For each of erlotinib, gefitinib, and afatinib, two trials showed improvement in one or more indices for the TKI compared to chemotherapy. The quality of evidence was high for the comparisons of erlotinib and gefitinib with cytotoxic chemotherapy and for the comparison of afatinib with cytotoxic chemotherapy.

# Erlotinib, gefitinib, afatinib and icotinib are all active agents in EGFR M+ NSCLC patients, and demonstrate an increased tumour response rate and prolonged PFS compared to cytotoxic chemotherapy. We found a beneficial effect of the TKI compared to cytotoxic chemotherapy in adverse effect and health-related quality of life. We found limited evidence for increased OS for the TKI when compared with standard chemotherapy, but the majority of the included trials allowed participants to switch treatments on disease progression, which will have a confounding effect on any OS analysis.

# Single agent-TKI remains the standard of care and the benefit of combining a TKI and chemotherapy remains uncertain as the evidence is based on small patient numbers. Cytotoxic chemotherapy is less effective in EGFR M+ NSCLC than erlotinib, gefitinib, afatinib or icotinib and is associated with greater toxicity. There are no data supporting the use of monoclonal antibody therapy. Icotinib is not available outside China."""


In [48]:
def rewrite_abstract(client_openai, abstract: str, conversation_text: str, sata_text: str) -> str:
    sata_text = format_sata(sata_text)
    system_prompt = (
        "CRITICAL RULE:\n"
        "If the user asks 'why' or 'how' and the abstract does not give a reason, you MUST include an explicit sentence saying the reason is unknown or not well understood.\n\n"

        "You are an expert science communicator.\n\n"
        "Your task is to rewrite the abstract into a personalized, plain-language summary for this specific reader.\n\n"
        "You MUST use the questions the user is confused about below to understand what the user is confused about or curious about, "
        "and make sure those topics are clearly explained in the rewritten abstract.\n\n"
        f"Questions:\n{conversation_text}\n\n"
        f"Select-All-That-Apply (SATA) Questions:\n{sata_text}\n\n"
        "For each SATA item:\n"
        "- The rewritten summary MUST contain information that allows a careful reader to logically deduce every correct answer.\n"
        "- You must NOT explicitly list, label, or reference answer choices or say which options are correct inside the summary.\n"
        "- The summary MUST avoid adding statements that would also justify incorrect options.\n"
        "- Add background knowledge only if it is necessary to answer a user question or enable SATA deduction.\n"
        "- The summary must remain natural narrative, not exam-style reasoning.\n\n"
        "Follow these steps internally (do NOT show them in your final answer):\n"
        "1. Identify every user question in the conversation text.\n"
        "2. For each user question:\n"
        "   - If the abstract contains the answer, explain it clearly using only abstract content.\n"
        "   - If not, add only the minimal well-established background needed to answer it.\n"
        "   - For any 'why' or 'how' question, include a causal explanation.\n"
        "   - If a user asks 'why' or 'how' and the abstract does not provide a reason, you MUST include an explicit sentence stating that the reason is unknown or not well understood.\n"
        "3. For each SATA question:\n"
        "   - Identify what facts allow correct options to be inferred.\n"
        "   - Embed those facts naturally in the summary.\n"
        "   - Ensure no incorrect option is supported.\n"
        "4. Final internal check:\n"
        "   - Every user question is answered by at least one sentence.\n"
        "   - No sentence exists that does not help answer a user question or support SATA deduction.\n"
        "   - No incorrect SATA option is supported.\n"
        "   - If any condition fails, rewrite until all pass.\n\n"
        "Final output rules:\n"
        "- Output only the final personalized plain-language summary.\n"
        "- Do NOT show reasoning steps, checklists, or internal notes.\n"
        "- If any user question is not clearly answered, the output is wrong.\n"
        "- Do not add information that does not serve answering user questions or enabling SATA deduction."
    )

    response = client_openai.chat.completions.create(
        model="gpt-4o",
        messages=[
            {"role": "system", "content": system_prompt},
            {"role": "user", "content": f"Rewrite this abstract:\n\n{abstract}"},
        ],
    )
    return response.choices[0].message.content.strip()


In [49]:
# personalized_summary = rewrite_abstract(client_openai, abstract, conversation_text, sata_text)
# print(personalized_summary)

In [50]:
# def generate_lay_questions_abstract(client_openai, abstract: str) -> str:
#     system_prompt = (
#         "You are a layperson with no scientific or medical background."
#         "Your task is to generate up to 10 questions that you would need answered"
#         "in order to understand the following abstract. "
#         "Output the questions as a single line, separated by semicolons."
#     )

#     response = client_openai.chat.completions.create(
#         model="gpt-4o",
#         messages=[
#             {"role": "system", "content": system_prompt},
#             {
#                 "role": "user",
#                 "content": f"Abstract:\n{abstract}\n\n"
#                            "Please list up to 10 questions, separated by semicolons."
#             },
#         ],
#     )
#     print(response.choices[0].message.content.strip())
#     return response.choices[0].message.content.strip()

In [51]:
abstract_df = pd.read_csv("new_abstract_dataset.csv")
# abstract_df["lay_person_question"] = abstract_df["abstract"].apply(lambda x: generate_lay_questions_abstract(client_openai, x))
# abstract_df.to_csv("new_abstract_dataset.csv")

In [52]:
import ast

def build_sata_from_row(row):
    chosen = row["chosen_questions"]
    if isinstance(chosen, str):
        chosen = ast.literal_eval(chosen)

    sata_text = []

    for q_num in chosen:
        q_key = f"q{q_num}"

        question = row[f"{q_key}_question"]

        options = []
        correct_answers = []

        for opt_i in [1, 2, 3]:
            opt_text = row[f"{q_key}_option{opt_i}_answer"]
            opt_correct = row[f"{q_key}_option{opt_i}_correct"]

            if pd.isna(opt_text):
                continue

            options.append(opt_text)
            if opt_correct is True or str(opt_correct).upper() == "TRUE":
                correct_answers.append(opt_text)

        sata_text.append({
            "question": question,
            "options": options,
            "correct_answers": correct_answers
        })

    return sata_text

In [ ]:
abstract_ids = ["1001", "1002", "1003", "1004", "1005", "1006", "1007", "1008", "1009", "1011"]
abstract_ids_set = set(abstract_ids)

abstract_df["generated_summary"] = None

for idx, row in abstract_df.iterrows():
    if str(row["abstract_id"]) not in abstract_ids_set:
        continue

    conversation_text = "\n".join(
        f"User: {q.strip()}"
        for q in row["lay_person_question"].split(";")
        if q.strip()
    )

    sata_text = build_sata_from_row(row)
    generated = rewrite_abstract(
        client_openai,
        abstract=row["abstract"],
        conversation_text=conversation_text,
        sata_text=sata_text
    )

    abstract_df.at[idx, "generated_summary"] = generated

abstract_df.to_csv("new_abstract_dataset.csv")

: 